In [9]:
# From master repository of the papper: [On the Fair Comparison of Optimization Algorithms in Different Machines](https://github.com/EtorArza/RTDHW)
import os
import sys
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath('../RTDHW-master'))
from equivalent_runtime import get_equivalent_runtime_from_probability

# Pre-processing input data

In [10]:
# test_df are created in eda notebook
test_df = pd.read_csv('csv/test_df.csv')

# Model from papper _On the Fair Comparison of Optimization Algorithms in Different Machines_

In [11]:
# Using a probability of predicting a longer than true equivalent runtime of p𝛾 = 0.01
# Get the RMSE for X_test
class RTDHWModel:
	def __init__(self, p=0.05):
		self.p = p
	
	def predict(self, X: pd.DataFrame):
		predictions = []
		for _, row in X.iterrows():
			prediction = get_equivalent_runtime_from_probability(self.p, row['score'], row['score_target'], row['total_time'])
			predictions.append(prediction)
		return np.array(predictions)

rtdhw_model = RTDHWModel()

# Describe Validation

In [12]:
def describe_val_RTDH(model: RTDHWModel, test_df: pd.DataFrame):
	min_instance = {"prediction": float('inf'), "actual": 0, "index": 0}
	max_instance = {"prediction": 0, "actual": 0, "index": 0}
	predictions = model.predict(test_df[['score', 'score_target','total_time']]).reshape(-1, 1)
	y = test_df['total_time_target'].values.reshape(-1, 1)
	index_min = np.argmin(np.abs(predictions - y))
	min_instance["prediction"] = predictions[index_min].item()
	min_instance["actual"] = y[index_min].item()
	min_instance["index"] = index_min
	index_max = np.argmax(np.abs(predictions - y))
	max_instance["prediction"] = predictions[index_max].item()
	max_instance["actual"] = y[index_max].item()
	max_instance["index"] = index_max

	return min_instance, max_instance, predictions

In [13]:
print("Validation set single thread model")
min_instance, max_instance, predictions = describe_val_RTDH(rtdhw_model, test_df)
y = test_df['total_time_target'].values.reshape(-1, 1)
errors = np.abs(predictions - y)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y)} | Std actual: {np.std(y)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(test_df.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(test_df.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model
Mean prediction: 18.916682593839692 | Std actual: 45.52009456013818
Mean actual: 23.631032844733987 | Std actual: 41.60096829835196
Mean Error: 8.623423887153386 | Std Error: 16.909485581919377
Min instance
total_time                                                             0.85
total_cpu_usage                                                        0.99
max_ram_usage                                                        21.875
brand_raw                          Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz
vendor_id_raw                                                  GenuineIntel
arch                                                                 X86_64
count                                                                    12
l2_cache_size                                                           1.5
l3_cache_size                                                          12.0
l2_cache_line_size                                                      25

In [14]:
# get index of total_time_target < 2.00
test_df_low = test_df[test_df['total_time_target'] < 2.0]
print("Validation set single thread model with total_time_target > 2.0")
min_instance, max_instance, predictions = describe_val_RTDH(rtdhw_model, test_df_low)
y = test_df_low['total_time_target'].values.reshape(-1, 1)
errors = np.abs(predictions - y)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y)} | Std actual: {np.std(y)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(test_df_low.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(test_df_low.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model with total_time_target > 2.0
Mean prediction: 0.47953822887086417 | Std actual: 0.3830120733292816
Mean actual: 0.6940603907637656 | Std actual: 0.5182334694492398
Mean Error: 0.2440584221983212 | Std Error: 0.2535843316830092
Min instance
total_time                                                             0.85
total_cpu_usage                                                        0.99
max_ram_usage                                                        21.875
brand_raw                          Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz
vendor_id_raw                                                  GenuineIntel
arch                                                                 X86_64
count                                                                    12
l2_cache_size                                                           1.5
l3_cache_size                                                          12.0
l2_cache_line_size                       

In [15]:
# get index of 2.00 > total_time_target > 150.0
test_df_med = test_df[test_df['total_time_target'] > 150.0]
print("Validation set single thread model with total_time_target > 150.0")
min_instance, max_instance, predictions = describe_val_RTDH(rtdhw_model, test_df_med)
y = test_df_med['total_time_target'].values.reshape(-1, 1)
errors = np.abs(predictions - y)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y)} | Std actual: {np.std(y)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(test_df_med.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(test_df_med.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model with total_time_target > 150.0
Mean prediction: 252.68449145573595 | Std actual: 95.17015044370658
Mean actual: 221.57692307692307 | Std actual: 38.88362856977631
Mean Error: 72.34315170423746 | Std Error: 51.32937732774585
Min instance
total_time                                                           202.65
total_cpu_usage                                                        0.99
max_ram_usage                                                          61.0
brand_raw                          Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
vendor_id_raw                                                  GenuineIntel
arch                                                                 X86_64
count                                                                    56
l2_cache_size                                                          70.0
l3_cache_size                                                          42.0
l2_cache_line_size                          

In [16]:
# get index of total_time_target > 150.00
test_df_high = test_df[test_df['total_time_target'] > 150.0]
print("Validation set single thread model with total_time_target > 150.0")
min_instance, max_instance, predictions = describe_val_RTDH(rtdhw_model, test_df_high)
y = test_df_high['total_time_target'].values.reshape(-1, 1)
errors = np.abs(predictions - y)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y)} | Std actual: {np.std(y)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(test_df_high.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(test_df_high.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model with total_time_target > 150.0
Mean prediction: 252.68449145573595 | Std actual: 95.17015044370658
Mean actual: 221.57692307692307 | Std actual: 38.88362856977631
Mean Error: 72.34315170423746 | Std Error: 51.32937732774585
Min instance
total_time                                                           202.65
total_cpu_usage                                                        0.99
max_ram_usage                                                          61.0
brand_raw                          Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
vendor_id_raw                                                  GenuineIntel
arch                                                                 X86_64
count                                                                    56
l2_cache_size                                                          70.0
l3_cache_size                                                          42.0
l2_cache_line_size                          